In [557]:
import pandas as pd
import nltk
import string
import re
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
from tqdm import tqdm
import emoji
from sklearn.metrics import *
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [558]:
def func_Eval(Y_Actual,Y_pred):

    TrueNegative=0
    TruePositive=0
    FalsePositive=0
    FalseNegative=0
    for i in range(len(Y_pred)):
        #if the actual class is negative
        if(Y_Actual[i]=="fake"):
            
            #if the predicited class is negative
            if(Y_pred[i]=="fake"):
                TrueNegative+=1
                
            #if the predicted class is positive
            else:
                FalsePositive+=1
                
        #if the actual class is positive
        else:
            #if the predicited class is positive
            if(Y_pred[i]=="real"):
                TruePositive+=1
                
            #if the predicited class is negative
            else:
                FalseNegative+=1

    Confusion_Matrix=[[TrueNegative,FalsePositive],[FalseNegative,TruePositive]]
    
    Confusion_Matrix=pd.DataFrame(Confusion_Matrix,columns=['Actual Negative','Actual Positive'])
    
    Confusion_Matrix.rename(index={0: "Predicted Negative", 1: "Predicted Positive"},inplace=True)
    
    MyPrecision=TruePositive/(TruePositive+FalsePositive)

    MyRecall=TruePositive/(TruePositive+FalseNegative)
    
    MyAccuracy=(TruePositive+TrueNegative)/(TruePositive+TrueNegative+FalseNegative+FalsePositive)
    
    MyF1score= 2*(MyPrecision*MyRecall)/(MyPrecision+MyRecall)
    
    MyPrecisionZero=TrueNegative/(TrueNegative+FalseNegative)

    MyRecallZero=TrueNegative/(TrueNegative+FalsePositive)
    
    
    MyF1scoreZero= 2*(MyPrecisionZero*MyRecallZero)/(MyPrecisionZero+MyRecallZero)

    
    print("-----------------")
    print("Accuracy:",MyAccuracy)
    print("-----------------")

    print()

    print("For the Positive CLass, Label:4")
    print("-----------------")
    print("Precision:",MyPrecision)

    print("-----------------")
    print("Recall:",MyRecall)

    print("-----------------")
    print("F1 Score:",MyF1score)

    print()

    print("For the Negative CLass, Label:0")

    print("-----------------")
    print("Precision:",MyPrecisionZero)

    print("-----------------")
    print("Recall:",MyRecallZero)

    print("-----------------")
    print("F1 Score:",MyF1scoreZero)

    print()
    print("--------")
    print("Macro Average F1",(MyF1score+MyF1scoreZero)/2)
    print("-----------------")
    print("Confusion Matrix:")
    print(Confusion_Matrix)

In [559]:
def preprocess_text_caller(Tweets_List):
    stop_words_list=stopwords.words('english')
    for i in (range(len(Tweets_List))):
        Tweets_List[i]=preprocess_text(Tweets_List[i],stop_words_list=stop_words_list)


In [560]:
def preprocess_text(tweet, flag_stemm=True, flag_lemm=True, stop_words_list=None):
#     print(tweet)
    tweet=emoji.demojize(tweet)
    tweet=tweet.replace(":"," ")
#     print(tweet)
#     tweet=p.clean(tweet)
    ## clean (convert to lowercase and remove punctuations and characters and then strip and remove url)
    tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
#     tweet = re.sub(r'19', 'Z', str(tweet).lower().strip())
#     tweet = re.sub(r'[0-9]+k*', ' X ', str(tweet).lower().strip())    
#     tweet = re.sub(r'[^\w\s]', '', str(tweet).lower().strip())
    ## Tokenize (convert from string to list) and remove the stop words
#     print(tweet)

    tokenize_tweet = tweet.split()

    if stop_words_list is not None:
        tokenize_tweet = [word for word in tokenize_tweet if word not in stop_words_list]


    ## Stemming (remove -ing, -ly, ...)
    if flag_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        tokenize_tweet = [ps.stem(word) for word in tokenize_tweet]

    ## Lemmatisation (convert the word into root word)
    if flag_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        tokenize_tweet = [lem.lemmatize(word) for word in tokenize_tweet]
#     print(tokenize_tweet)
    ## back to string from list
    tweet = " ".join(tokenize_tweet)
    return tweet

In [561]:
def getLength(GivenTweets):
    ret=np.zeros(len(GivenTweets))
    for i in (range(len(GivenTweets))):
        tweet=GivenTweets[i]
        ret[i] = len(tweet)
    return ret
def getAvgWord(GivenTweets):
    
    ret=np.zeros(len(GivenTweets))
    for i in (range(len(GivenTweets))):
        tweet=GivenTweets[i].split()
        cnt = 0
        add = 0
        for j in range(len(tweet)):
            cnt+=1
            add+= len(tweet[j])
        ret[i] = add/cnt
    return ret
def getSpecial(GivenTweets):
    ret=np.zeros(len(GivenTweets))
    for i in (range(len(GivenTweets))):
        tweet=GivenTweets[i].split()
        cnt = 0
        for j in range(len(tweet)):
            if(tweet[j]=='X'):
                cnt+=1
        ret[i] = cnt
    return ret
    
def vader( GivenTweets):
    obj = SentimentIntensityAnalyzer()
    Comp_vader = np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
        sentiment_dict = obj.polarity_scores(GivenTweets[i])
        Comp_vader[i]=sentiment_dict['compound']
    return(Comp_vader)
def FuncAllCapsCount( GivenTweets):
    # Function to get All caps count from the tweets
    # pass tweets as it is

    AllCapsCount = np.zeros(len(GivenTweets))
    for i in range(len(GivenTweets)):
        tweet = nltk.word_tokenize(GivenTweets[i])
        for word in tweet:
            if(word!="I" and word != "USER" and word != "URL" and re.match("^[A-Z]+$",word)):
                AllCapsCount[i] += 1

    return AllCapsCount


In [562]:
df_training=pd.DataFrame(pd.read_csv("Constraint_English_Train - Sheet1.csv"))
df_training.drop(columns="id",inplace=True)
# df_training
df_training['tweet_original']=df_training['tweet']
preprocess_text_caller(df_training['tweet'])

In [563]:
df_validation=pd.DataFrame(pd.read_csv("Constraint_English_Val - Sheet1.csv"))
df_validation.drop(columns="id",inplace=True)
# df_validation
df_validation['tweet_original']=df_validation['tweet']
preprocess_text_caller(df_validation['tweet'])

In [564]:
df_test=pd.DataFrame(pd.read_csv("Constraint_English_Test - Sheet1.csv"))
# df_test
df_test['tweet_original']=df_test['tweet']
preprocess_text_caller(df_test['tweet'])

<ipython-input-559-fb9ae317fd00>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Tweets_List[i]=preprocess_text(Tweets_List[i],stop_words_list=stop_words_list)


In [576]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=2)

In [577]:
# Train_ngrams=vectorizer.fit_transform(df_training.tweet)
vectorizer.fit_transform(list(df_training.tweet) + list(df_validation.tweet) +list(df_test.tweet))
Train_ngrams=vectorizer.transform(df_training.tweet)

In [578]:
Validation_f=pd.DataFrame(FuncAllCapsCount(df_validation.tweet))


In [579]:
Validation_ngrams=vectorizer.transform(df_validation.tweet)

In [580]:
Validation_ngrams

<2140x28361 sparse matrix of type '<class 'numpy.float64'>'
	with 50472 stored elements in Compressed Sparse Row format>

In [581]:
Test_ngrams=vectorizer.transform(df_test.tweet)

In [582]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
clf.fit(Train_ngrams,df_training["label"])
y_pred_rf=clf.predict(Validation_ngrams)


In [583]:
f1_score(y_pred_rf,df_validation["label"],average="macro")

0.9214293769160768

In [584]:
# f1_score(y_pred,df_validation["label"],average="macro")
# func_Eval(y_pred,df_validation["label"])

In [585]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
clf.fit(Train_ngrams,df_training["label"])
y_pred_svm=clf.predict(Validation_ngrams)


In [586]:
f1_score(y_pred_svm,df_validation["label"],average="macro")

0.9420009074925841

In [530]:
# func_Eval(y_pred_svm,df_validation["label"])

In [587]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(Train_ngrams,df_training["label"])
y_pred_log=clf.predict(Validation_ngrams)


In [588]:
f1_score(y_pred_log,df_validation["label"],average="macro")

0.9199220797238565

In [535]:
# func_Eval(y_pred_log,df_validation["label"])

In [589]:
from sklearn.neural_network import MLPClassifier
clf2 = MLPClassifier(random_state=1, max_iter=300,verbose=True,activation="logistic",solver="lbfgs")
clf2.fit(Train_ngrams,df_training["label"])
y_pred_mlp=clf2.predict(Validation_ngrams)
f1_score(y_pred_mlp,df_validation["label"],average="macro")

0.9302788169483306

In [595]:
from sklearn.ensemble import BaggingClassifier
svc = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
bag = BaggingClassifier(svc,n_estimators=50, random_state=0,n_jobs=-1)
bag.fit(Train_ngrams,df_training["label"])
y_pred_bag=bag.predict(Validation_ngrams)
f1_score(y_pred_bag,df_validation["label"],average="macro")

0.9420163689695801

In [592]:
y=[]

for i in range(len(y_pred_log)):
    if(y_pred_svm[i]==y_pred_mlp[i]):
        y.append(y_pred_svm[i])
    elif(y_pred_svm[i]==y_pred_rf[i]):
        y.append(y_pred_svm[i])
    elif(y_pred_rf[i]==y_pred_mlp[i]):
        y.append(y_pred_log[i])












In [593]:
f1_score(y,df_validation["label"],average="macro")

0.9354368180661479

In [597]:

import requests

shorturl = input("Enter the shortened URL in its entirety: ")
r = requests.get(shorturl)

print("""
The shortened URL forwards to:

    %s
""" % r.url)


Enter the shortened URL in its entirety: https://t.co/swClEhi2Iq

The shortened URL forwards to:

    https://www.thedailymash.co.uk/news/health/kids-reach-fk-this-shit-stage-of-lockdown-20200612197394



In [ ]:
websites=[]

In [ ]:
for i in df_training.tweet:
    re.match("")

In [541]:
# y_pred=clf.predict(Validation_ngrams)
# f1_score(df_validation["label"],y_pred,average="macro")

In [542]:
# y_pred_test=clf.predict(Test_ngrams)

In [543]:
# y_pred_test=pd.DataFrame(y_pred_test)

In [544]:
# ans=pd.DataFrame(df_test["id"])
# ans["label"]=y_pred_test
# ans.to_csv("answer9349valid.txt",index=False)

In [545]:
# count=0
# string=""
# for i in range(len(ans_sudhir.label)):
#     if not((ans_sudhir.label[i]==ans.label[i]) and (ans.label[i] ==ans_MLP.label[i])):
#         print("*************")
#         print(ans.id[i],ans_sudhir.label[i],ans.label[i],ans_MLP.label[i])
#         print(df_test.tweet_original[i])
#         print("*************")
# #         string+="\n"+"\n" "*************"+"\n "+str(ans.id[i])+" Sudhir:"+str(ans_sudhir.label[i])+" Navneet:"+ans.label[i]+"\n"+df_test.tweet_original[i]
#         count+=1

In [546]:
# count

In [547]:
# file=open("mismatch.txt","w")
# file.write(string)
# file.close()

In [548]:
# ans.label.value_counts()

0.9316940249105621

In [474]:
y_pred_test_MLP=clf2.predict(Test_ngrams)

In [496]:
ans_MLP=pd.DataFrame(df_test["id"])
ans_MLP["label"]=y_pred_test_MLP
# ans.to_csv("answer9349valid.txt",index=False)

,id,label
0,1,real
1,2,fake
2,3,fake
3,4,real
4,5,fake
...,...,...
2135,2136,real
2136,2137,fake
2137,2138,real
2138,2139,real


In [65]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 23 13:54:19 2020

@author: suranjana
"""




from pysbd.utils import PySBDFactory
import urllib
import requests, re, spacy
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import numpy as np
import csv
import preprocessor as p
from gensim.parsing.preprocessing import remove_stopwords
from googletrans import Translator

translator = Translator()
ua = UserAgent()
nlp = spacy.load("en_core_web_sm")
# f = open('CommonWords.txt')
# commonEngWords = f.read().splitlines()
# f.close()
popular_links = [
        "nytimes", "wsj", "huffpost", "washingtonpost","time","republicworld",
        "latimes", "reuters", "abcnews", "usatoday",
        "bloomberg", "nbcnews", "dailymail", "theguardian",
        "thesun", "mirror", "telegraph", "bbc",
        "thestar", "theglobeandmail", "forbes",
        "cnbc", "chinadaily", "nypost", "usnews",
        "timesofindia", "thehindu", "hindustantimes",
        "cbsnews", "sfgate", "thehill", "thedailybeast",
        "newsweek", "theatlantic", "nzherald", "vanguardngr",
        "dailysun", "thejakartapost", "thestar",
        "straitstimes", "bangkokpost", "japantimes",
        "thedailystar", "scmp", "yahoo.com/news", "news.google"
        ]


def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])





def get_search_result(query, number_result = 20):
    query = urllib.parse.quote_plus(query) # Format into URL encoding
    
    
    google_url = "https://www.google.com/search?q=" + query + "&num=" + str(number_result)
    response = requests.get(google_url, {"User-Agent": ua.random})
    soup = BeautifulSoup(response.text, "html.parser")
    
    result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})
    
    links = []
    titles = []
    descriptions = []
    for r in result_div:
        # Checks if each element is present, else, raise exception
        try:
            link = r.find('a', href = True)
            title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
            description = r.find('div', attrs={'class':'s3v9rd'}).get_text()
            
            # Check to make sure everything is present before appending
            if link != '' and title != '' and description != '': 
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)
        # Next loop if one element is not present
        except:
            continue
    return links, titles, descriptions

 
def clean_links(links, titles, descriptions):
    to_remove = []
    clean_links = []
    for i, l in enumerate(links):
        clean = re.search('\/url\?q\=(.*)\&sa',l)
    
        # Anything that doesn't fit the above pattern will be removed
        if clean is None:
            to_remove.append(i)
            continue
        clean_links.append(clean.group(1))
    
    # Remove the corresponding titles & descriptions
    for x in to_remove:
        del titles[x]
        del descriptions[x]
    return clean_links, titles, descriptions


def filter_links(links, titles, descriptions):
    to_remove = []
    for i, l in enumerate(links):
        if not any(a in l for a in popular_links):
            to_remove.append(i)
            continue
    
    # Remove the corresponding titles & descriptions
    links = [l for i,l in enumerate(links) if i not in to_remove]
    titles = [t for i,t in enumerate(titles) if i not in to_remove]
    links = [l for i,d in enumerate(descriptions) if i not in to_remove]
    return links, titles, descriptions
    

def valid_links(links):
    no_link = 0
    for link in links:
        if any(implink in link for implink in popular_links):
            no_link = no_link+1
    return no_link/len(links)


def split_sentence(text):
    nlp = spacy.blank('en')
    nlp.add_pipe(PySBDFactory(nlp))
    doc = nlp(text)
    return([sent.text for sent in doc.sents if sent.text.isspace()==False])


def valid_description(query, descriptions):
    temp_query = preprocessing(query)
    sentences = split_sentence(temp_query)
    score = []
    for q in query:
        temp = [levenshtein_ratio_and_distance(q, sent,ratio_calc = True) for sent in sentences]
        score.append(max(temp))
    return sum(score)/len(score)


def preprocessing(text):
    p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())    
    text = p.clean(text)
    text = remove_stopwords(text)
    text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ').replace('@','').replace('#','. ').replace('&amp;', 'and')
    return text


def translate_text(text):
    result = translator.translate('Mitä sinä teet')
    return result.text

    
# if __name__ == "__main__":
#     query = "'trade war'"
#     threshold = 0.3
    tweets = []
    clean_tweets = []
    labels = []
    with open('en_dataset.csv', 'rt') as csv_file:
        csv_reader = csv.reader(csv_file)
        line_count = 0
        for row in csv_reader:
            if line_count > 0:
                tweets.append(row[2])
                clean_tweets.append(preprocessing(row[2]))
                labels.append(row[3])
            line_count += 1
#     score = []
#     for query in tweets:
#         links, titles, descriptions = get_search_result(query)
#         links, titles, descriptions = filter_links(links, titles, descriptions)
#         #descriptions = [preprocessing(desc) for desc in descriptions]
#         if len(titles)>0:
#             titles = [preprocessing(title.lower()) for title in titles]
#             desc_score = valid_description(query, titles)
#             score.append(desc_score)
#         else:
#             score.append(0)
    
    


In [68]:
text="i am a good boy"
# p.set_options(p.OPT.URL, p.OPT.RESERVED, p.OPT.EMOJI, p.OPT.SMILEY)
text = re.sub(r'[^\w\s]', '', str(text).lower().strip())    
text = p.clean(text)
text = remove_stopwords(text)
text = text.lower().replace('[^\w\s]',' ').replace('\s\s+', ' ').replace('@','').replace('#','. ').replace('&amp;', 'and')
text

'good boy'

In [53]:
# clean_tweets

In [85]:
score=[]
for query in tqdm(df_training.tweet_original[96:]):
    links, titles, descriptions = get_search_result(query)
    links, titles, descriptions = filter_links(links, titles, descriptions)
    #descriptions = [preprocessing(desc) for desc in descriptions]

    if len(titles)>0:
        titles = [preprocessing(title.lower()) for title in titles]
        desc_score = valid_description(query, titles)
        score.append(desc_score)
    else:
        score.append(0)

  8%|▊         | 533/6324 [07:52<1:25:32,  1.13it/s]


KeyboardInterrupt: 

AttributeError: 'DataFrame' object has no attribute 'label'

In [83]:
for i in range(len(score[0:95])):
#     print(i,score[i],df_training.label[i],s[i])
        print(i,score[i])

0 0.0125479261066574
1 0.016719866241070046
2 0.015267175572519082
3 0.010056925996204944
4 0.009802458078320187
5 0.010391634734039156
6 0
7 0.019974635383639836
8 0.03090909090909085
9 0
10 0
11 0.011405901314158228
12 0
13 0.011572742955721709
14 0
15 0.0060990099009901036
16 0.012350119904076693
17 0.007973311092577129
18 0.012672176308539935
19 0
20 0
21 0
22 0.01397646069777217
23 0.01258414617392501
24 0.017663639210773564
25 0.01753312945973494
26 0
27 0
28 0.010941331824184051
29 0.011924119241192426
30 0.008901651112706379
31 0.018849206349206324
32 0.010223478037503184
33 0.025717566016073442
34 0
35 0.016246684350132608
36 0
37 0.03361344537815128
38 0.023953823953823936
39 0.021733073279465005
40 0
41 0.01848739495798316
42 0.012285927029039502
43 0.01005544980551189
44 0.01993769470404981
45 0.015142337976983628
46 0.018779342723004706
47 0.01143168209036472
48 0.014456316781898201
49 0.013580194846273864
50 0.00981402986214539
51 0.00919305413687438
52 0.0287807430664573

In [111]:
lis=score2+score+score3
# len(score3)+len(score2)+len(score)
# len(lis)
# len(score3)

In [84]:

file=open("score_train_95",'wb')
pickle.dump(score,file)
file.close()

In [72]:
import pickle
file=open("score_train",'rb')
s=pickle.load(file)
file.close()
len(s)

6420

In [137]:
train_lis=[]
for i in lis:
        train_lis.append([i])
valid_lis=[]
for i in score:
        valid_lis.append([i])


In [199]:
from scipy.sparse import csr_matrix, hstack

csr_TrainFeature_without_ngram=csr_matrix(Train_f)
csr_ValidFeature_without_ngram=csr_matrix(Validation_f)
TrainFeatureFinal=hstack([Train_ngrams,csr_TrainFeature_without_ngram])
ValidFeatureFinal=hstack([Validation_ngrams,csr_ValidFeature_without_ngram])

In [200]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto',kernel="sigmoid"))
clf.fit(TrainFeatureFinal,df_training["label"])
y_pred=clf.predict(ValidFeatureFinal)

In [201]:
f1_score(y_pred,df_validation["label"],average="macro")

0.9223158467450243

In [191]:
func_Eval(y_pred,df_validation["label"])

-----------------
Accuracy: 0.9247663551401869
-----------------

For the Positive CLass, Label:4
-----------------
Precision: 0.9205357142857142
-----------------
Recall: 0.9347234814143246
-----------------
F1 Score: 0.9275753486279802

For the Negative CLass, Label:0
-----------------
Precision: 0.9294117647058824
-----------------
Recall: 0.914175506268081
-----------------
F1 Score: 0.921730675741371

--------
Macro Average F1 0.9246530121846757
-----------------
Confusion Matrix:
                 Predicted Negative  Predicted Positive
Actual Negative                 948                  89
Actual Positive                  72                1031
